In [115]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import itertools

from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.preprocessing import StandardScaler

In [32]:
class CustomDataset(Dataset):
    
    def __init__(self, filepath) -> None:
        
        # load csv data
        data = pd.read_csv(filepath, header=None)
        X = data.iloc[:, :-1].values
        y = data.iloc[:, -1].values
        
        # feature scaling
        sc = StandardScaler()
        X = sc.fit_transform(X)
        
        # convert to tensors
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [33]:
dataset = CustomDataset('train_all_0.csv')

# create data indices for train val split
data_size = len(dataset)
indices = list(range(data_size))
split = int(np.floor(0.2 * data_size))
np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

# create data loader
train_loader = DataLoader(dataset, batch_size=16, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size=16, sampler=val_sampler)

In [34]:
class BinaryClassification(nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()
        self.layer_1 = nn.Linear(12, 128) 
        self.layer_out = nn.Linear(128, 2) 
        self.relu = nn.ReLU()
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.layer_out(x)
        
        return x

In [121]:
# create training pipeline based on page 49
def P49_train(train_loader=train_loader, val_loader=val_loader, model=None, epochs=None, criterion=None, optimizer=None, loss_threshold=0.5):
    '''When iterating over 50 or validation loss smaller than specific number, the function will stop'''
    for epoch in range(epochs):
        
        model.train()
        train_loss = []
        train_accs = []
        
        for batch in train_loader:
            
            x, y = batch
            
            logits = model(x.to(device))
            loss = criterion(logits, y.to(device))
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            acc = (logits.argmax(dim=-1) == y.to(device)).float().mean()
            train_loss.append(loss.item())
            train_accs.append(acc)
        
        train_loss = sum(train_loss) / len(train_loss)
        train_acc = sum(train_accs) / len(train_accs)
        
        model.eval()

        valid_loss = []
        valid_accs = []
        
        for batch in val_loader:
            imgs, labels = batch
            
            with torch.no_grad():
                logits = model(imgs.to(device))
                
                acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
                
                valid_loss.append(loss.item())
                valid_accs.append(acc)
        
        valid_loss = sum(valid_loss) / len(valid_loss)
        valid_acc = sum(valid_accs) / len(valid_accs)
        
        print(f'[ {epoch+1}/{epochs} ] | train_loss = {train_loss:.5f}, train_acc = {train_acc:.5f}, val_loss = {valid_loss:.5f}, val_acc = {valid_acc:.5f}')
        
        if epoch+1 >= 50:
            print("It's over 50 epochs, stop training")    
            break
        
        if train_loss < loss_threshold:
            print('The training loss is smaller than what you want, stop training')
            break
            

In [109]:
# create training pipeline based on page 50
def P50_train(train_loader=train_loader, val_loader=val_loader, model=None, epochs=None, criterion=None, optimizer=None, loss_threshold=0.5, eta_threshold=0.008):
    '''
    When iterating over 50 or validation loss smaller than specific number, the function will stop.
    Also, if the new loss is larger than the previous one, it will compare the learning rate and the threshold.
    if the learning rate is larger than the threshold, use the old weight and the learning rate will multiply 0.7 and do bp one more time.
    if not, then stop training.
    However if the new loss is smaller than the previous one, update w to new weight and multiply learning rate by 1.2, and going through new loop
    '''
    
    previous_train_loss = 10000    

    for epoch in range(epochs):
        
        model.train()
        
        previous_model_params = model.state_dict()
        stop_training = False
        
        while optimizer.param_groups[0]['lr'] > eta_threshold:
            
            train_loss = []
            train_accs = []
            
            for batch in train_loader:
                
                x, y = batch
                
                logits = model(x.to(device))
                loss = criterion(logits, y.to(device))
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                acc = (logits.argmax(dim=-1) == y.to(device)).float().mean()
                train_loss.append(loss.item())
                train_accs.append(acc)
            
            train_loss = sum(train_loss) / len(train_loss)
            train_acc = sum(train_accs) / len(train_accs)
            
            if train_loss < previous_train_loss:
                optimizer.param_groups[0]['lr'] *= 1.2
                previous_train_loss = train_loss
                print(f'The previous training loss is: {previous_train_loss}')
                break
            
            optimizer.param_groups[0]['lr'] *= 0.7
            model.load_state_dict(previous_model_params)
            current_lr = optimizer.param_groups[0]['lr']
            print(f'lr shrinking!, now the lr is: {current_lr}')
            
        else:
            stop_training = True
        
        # Use try and except to detect whether the eta_threshold is set too high initially
        try:        
            model.eval()
            valid_loss = []
            valid_accs = []
            
            for batch in val_loader:
                imgs, labels = batch
                
                with torch.no_grad():
                    logits = model(imgs.to(device))
                    
                    acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
                    valid_loss.append(loss.item())
                    valid_accs.append(acc)
            
            valid_loss = sum(valid_loss) / len(valid_loss)
            valid_acc = sum(valid_accs) / len(valid_accs)
        
            print(f'[ {epoch+1}/{epochs} ] | train_loss = {train_loss:.5f}, train_acc = {train_acc:.5f}, val_loss = {valid_loss:.5f}, val_acc = {valid_acc:.5f}')
            
        except UnboundLocalError:
            print('Your eta_threshold is setting higher than your learning rate. Reset it with lower one!')
        
        # stopping criterion
        if stop_training:
            print('Learning rate is smaller than the threshold, stop training.')
            break
        
        if epoch+1 >= 50:
            print('It over 50 epochs, stop training.')
            break
        
        if train_loss < loss_threshold:
            print('The training loss is smaller than what you want, stop training.')
            break

In [118]:
# create training pipeline based on page 58
def P58_train(train_loader=train_loader, val_loader=val_loader, model=None, criterion=None, optimizer=None, loss_threshold=0.5, eta_threshold=0.008):
    '''
    Almost the same as P50, but without training epochs.
    '''
    
    previous_train_loss = 10000    

    for epoch in itertools.count():
        
        model.train()
        
        previous_model_params = model.state_dict()
        stop_training = False
        
        while optimizer.param_groups[0]['lr'] > eta_threshold:
            
            train_loss = []
            train_accs = []
            
            for batch in train_loader:
                
                x, y = batch
                
                logits = model(x.to(device))
                loss = criterion(logits, y.to(device))
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                acc = (logits.argmax(dim=-1) == y.to(device)).float().mean()
                train_loss.append(loss.item())
                train_accs.append(acc)
            
            train_loss = sum(train_loss) / len(train_loss)
            train_acc = sum(train_accs) / len(train_accs)
            
            if train_loss < previous_train_loss:
                optimizer.param_groups[0]['lr'] *= 1.2
                previous_train_loss = train_loss
                print(f'The previous training loss is: {previous_train_loss}')
                break
            
            optimizer.param_groups[0]['lr'] *= 0.7
            model.load_state_dict(previous_model_params)
            current_lr = optimizer.param_groups[0]['lr']
            print(f'lr shrinking!, now the lr is: {current_lr}')
            
        else:
            stop_training = True
        
        # Use try and except to detect whether the eta_threshold is set too high initially
        try:        
            model.eval()
            valid_loss = []
            valid_accs = []
            
            for batch in val_loader:
                imgs, labels = batch
                
                with torch.no_grad():
                    logits = model(imgs.to(device))
                    
                    acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
                    valid_loss.append(loss.item())
                    valid_accs.append(acc)
            
            valid_loss = sum(valid_loss) / len(valid_loss)
            valid_acc = sum(valid_accs) / len(valid_accs)
        
            print(f'[ {epoch+1}/{epochs} ] | train_loss = {train_loss:.5f}, train_acc = {train_acc:.5f}, val_loss = {valid_loss:.5f}, val_acc = {valid_acc:.5f}')
            
        except UnboundLocalError:
            print('Your eta_threshold is setting higher than your learning rate. Reset it with lower one!')
        
        # stopping criterion
        if stop_training:
            print('Learning rate is smaller than the threshold, stop training.')
            break
        
        if train_loss < loss_threshold:
            print('The training loss is smaller than what you want, stop training.')
            break

In [124]:
epochs = 100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BinaryClassification().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3)
P49_train(train_loader, val_loader, model, epochs, criterion, optimizer, loss_threshold=0.2)

[ 1/100 ] | train_loss = 0.54745, train_acc = 0.79427, val_loss = 0.34997, val_acc = 0.79167
[ 2/100 ] | train_loss = 0.42227, train_acc = 0.83073, val_loss = 0.42259, val_acc = 0.82292
[ 3/100 ] | train_loss = 0.36963, train_acc = 0.85677, val_loss = 0.32141, val_acc = 0.82292
[ 4/100 ] | train_loss = 0.34281, train_acc = 0.85677, val_loss = 0.29560, val_acc = 0.84375
[ 5/100 ] | train_loss = 0.32344, train_acc = 0.86719, val_loss = 0.22710, val_acc = 0.84375
[ 6/100 ] | train_loss = 0.30918, train_acc = 0.87500, val_loss = 0.41587, val_acc = 0.84375
[ 7/100 ] | train_loss = 0.29792, train_acc = 0.86979, val_loss = 0.13412, val_acc = 0.86458
[ 8/100 ] | train_loss = 0.29051, train_acc = 0.88281, val_loss = 0.23659, val_acc = 0.87500
[ 9/100 ] | train_loss = 0.28087, train_acc = 0.88281, val_loss = 0.38199, val_acc = 0.86458
[ 10/100 ] | train_loss = 0.27050, train_acc = 0.89323, val_loss = 0.24689, val_acc = 0.87500
[ 11/100 ] | train_loss = 0.26381, train_acc = 0.89844, val_loss = 0.

In [113]:
epochs = 100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BinaryClassification().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3)
P50_train(train_loader, val_loader, model, epochs, criterion, optimizer, loss_threshold=0.05, eta_threshold=9e-4)

The previous training loss is: 0.508815407132109
[ 1/100 ] | train_loss = 0.50882, train_acc = 0.80729, val_loss = 0.24727, val_acc = 0.79167
The previous training loss is: 0.4118443491558234
[ 2/100 ] | train_loss = 0.41184, train_acc = 0.82552, val_loss = 0.26101, val_acc = 0.82292
The previous training loss is: 0.35450122070809204
[ 3/100 ] | train_loss = 0.35450, train_acc = 0.85677, val_loss = 0.30807, val_acc = 0.85417
The previous training loss is: 0.32634108513593674
[ 4/100 ] | train_loss = 0.32634, train_acc = 0.86719, val_loss = 0.22383, val_acc = 0.88542
The previous training loss is: 0.3105933458233873
[ 5/100 ] | train_loss = 0.31059, train_acc = 0.88802, val_loss = 0.32867, val_acc = 0.89583
The previous training loss is: 0.2868718837077419
[ 6/100 ] | train_loss = 0.28687, train_acc = 0.90104, val_loss = 0.09822, val_acc = 0.87500
The previous training loss is: 0.27396557324876386
[ 7/100 ] | train_loss = 0.27397, train_acc = 0.90104, val_loss = 0.29306, val_acc = 0.895

In [125]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BinaryClassification().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3)
P58_train(train_loader, val_loader, model, criterion, optimizer, loss_threshold=0.05, eta_threshold=9e-4)

The previous training loss is: 0.6209744065999985
[ 1/100 ] | train_loss = 0.62097, train_acc = 0.66667, val_loss = 0.43850, val_acc = 0.79167
The previous training loss is: 0.44926222041249275
[ 2/100 ] | train_loss = 0.44926, train_acc = 0.81250, val_loss = 0.48079, val_acc = 0.81250
The previous training loss is: 0.37369591121872264
[ 3/100 ] | train_loss = 0.37370, train_acc = 0.85677, val_loss = 0.38961, val_acc = 0.84375
The previous training loss is: 0.3326238188892603
[ 4/100 ] | train_loss = 0.33262, train_acc = 0.87500, val_loss = 0.27778, val_acc = 0.86458
The previous training loss is: 0.30780113612612087
[ 5/100 ] | train_loss = 0.30780, train_acc = 0.86979, val_loss = 0.28548, val_acc = 0.87500
The previous training loss is: 0.2896661659081777
[ 6/100 ] | train_loss = 0.28967, train_acc = 0.87760, val_loss = 0.28282, val_acc = 0.87500
The previous training loss is: 0.27271145830551785
[ 7/100 ] | train_loss = 0.27271, train_acc = 0.89323, val_loss = 0.35666, val_acc = 0.8